# Analysing ChIP-seq data with BioNumPy

In this task, we will use BioNumPy to analyse the transcription factor CTCF. We will work with a publically available dataset from the Encode Project.

The task is divided into five parts:

1) Filtering/analysing raw FASTQ reads
2) Reading peaks (bed-files)
3) Working with reference genomes and extracting sequences
4) Motif matching and PWMs
5) Bonus-task: Find the unknown TF that cobinds with CTCF

In this part we also need the `jaspar` Python package for fetching motif PWMs from the Jaspar database, and the `Plotly` plotting package. Start by installing them:

In [ ]:
!pip install jaspar
!pip install plotly

We import everything we need:

In [ ]:
import bionumpy as bnp
import numpy as np
import plotly.express as plx
from pyjaspar import jaspardb

## Part 1: FASTQ-files
We download raw FASTQ reads from ENCODE (this is the same dataset as used in the intro exercises, so you can skip the download if you already have the data):

In [ ]:
!wget https://www.encodeproject.org/files/ENCFF001HTO/@@download/ENCFF001HTO.fastq.gz

Use BioNumPy and what you have learned until now to briefly analyse the raw reads. For instance, try to find the following (or other things you are interested in):

* The total number of reads
* The total number of bases in the reads
* The average read length
* The total number of G and C
* The GC-content
* The average base quality

Feel free to look at code from the previous part if you've forgotten how to do things.

## Part 2: Reading bed-files

Assume we have used the FASTQ-reads from the previous part to perform peak calling of the CTCF trancsription factor (e.g. using the MACS 2 peak caller).

We have obtained a bed file with the called peaks. This file contains intervals on a reference genome. We also need the hg38 reference genome sequence:

In [ ]:
#Download the bed file
!wget https://www.encodeproject.org/files/ENCFF843VHC/@@download/ENCFF843VHC.bed.gz
# Download the hg38 reference genome and unzip it
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
!gunzip hg38.fa.gz

**TASK**

Open and read the bed file using BioNumPy. Find out which fields your dataset has. NOTE: The bed file is quite small and we can fit it all into memory. Thus we can use `.read()` instead of reading single chunks:

In [ ]:
# your code here
file = # .. open the file
peaks = file.read()

** TASK **

* How many peaks are there on chromosome 14?
* What is the average peak size? (Hint: `peaks.stop-peaks.start` works)

In [ ]:
# You can write code here

## Part 3: Working with reference genomes and combining data sets

We want to be able to extract the reference genome sequence from each peak so that we can analyse motif patterns within peaks. BioNumPy supports reading and indexing of reference genome files quite efficiently:

In [ ]:
reference_genome = bnp.open_indexed("hg38.fa")
chromosome5_sequence = reference_genome["chr5"]
some_region_sequence = reference_genome["chr5"][12345:12360]
print(some_region_sequence)

PS: If an index file (hg38.fa.fai) does not exist, BioNumPy will create it the first time you open the reference genome. Thus, the above code may take a couple of minutes to run the first time you run it.

**TASK**

How many Gs are there in chromosome 1 of hg38? What is the GC-content in chromosome 1?

In [ ]:
# You can write code here

We've seen that we can extract sequences from single regions using simple lookup-like syntax. However, if we want to analyse the sequences of our 40k+ CTCF peaks, we cannot do this by querying one sequence at a time. Instead, BioNumPy lets us efficiently get all the reference genome sequences from all our peaks:

In [ ]:
contig_lengths = reference_genome.get_contig_lengths()
sorted_peaks = bnp.arithmetics.sort_all_intervals(peaks, sort_order=list(contig_lengths.keys()))
multistream = bnp.MultiStream(contig_lengths,
                              intervals=sorted_peaks,
                              sequence=reference_genome)
sequence_stream = bnp.sequence.get_sequences(multistream.sequence,
                                             multistream.intervals)
sequences = np.concatenate(list(sequence_stream))

We now have all the sequences of all the peaks efficiently stored in a NumPy-like data structure (EncodedRaggedArray). We can thus use BioNumPy to analyse these sequences.

We know that the CTCF transcription factor often binds to `CCCTC`. Remember from the previous exercise that we can get a boolean mask of where a sequence is matching like this:

In [ ]:
matches = bnp.sequence.string_matcher.match_string(sequences, "CCCTC")

`matches` is a RaggedArray, with one row per sequence. We would expect CTCF to bind close to the center of the peak. Thus, it could be interesting to find the position of each match within each peak and plot the positions as a histogram:

In [ ]:
row_positions, column_position = np.nonzero(matches)
fig = plx.histogram(column_position)
fig.show()

If you are able to generate the plot using the code above, you should see a distinct enrichment at around 130 bp (which is typically in the middle of most peaks).

**TASK**

* Generate the plot for another "random" sequence, such as "GAGAGA". Does the plot still show a distinct peak at the center?
* What is the GC-content inside the peak sequences? Is it higher than the GC-content of e.g. chromosome 1?


## Part 4: Motif matching

In the previous exercise, we looked for a very simple sequence pattern `CCCTC`. However, the preference that a transcription factor has to bind to a sequence can be represented better using a Position Weight Matrix (PWM). Such PWMs have been computed for many common transcription factors, and are available through the Jaspar Database.

We can get the PWM for CTCF from Jaspar like this:

In [ ]:
from bionumpy.sequence.position_weight_matrix import PWM, get_motif_scores
jaspar_object = jaspardb(release="JASPAR2020")
ctcf_motif = jaspar_object.fetch_motifs_by_name('CTCF')[0] # we pick the first of possible hits
ctcf_pwm = PWM.from_dict(ctcf_motif.pwm)
print(ctcf_pwm)

We can compute the score of this motif against all the sequences and get an estimate of how likely it is to bind:

In [ ]:
motif_scores = get_motif_scores(sequences, ctcf_pwm)
# we adjust the score for sequence length
adjusted_scores = motif_scores-np.logaddexp(motif_scores, ctcf_motif.length*np.log(0.25))

# assume that a score higher than 0.9 means that CTCF is likely to bind
sites = adjusted_scores > np.log(0.9)
print(sites)

The `sites` variable is now a boolean mask with True where we have predicted that the motif will bind.

**TASK**

* How many peaks have at least one predicted binding site?
* Also compute the number as a ratio of the total number of peaks

## Part 5: Bonus task

We hyphotesize that there is another transcription factor that often bind to the same regions as CTCF. Since we are able to get the motifs of other transcription factors, we can try to do the same analysis as above with motifs of other transcription factors and see if any seem to give high scores inside CTCF peaks.

* Fetch motifs for other transcription factors from Jaspar (see code below)
* For each motif, find the ratio of peaks with a match
* Try to see if one of the motifs has a high ratio of matches

Start by fetching a bunch of motifs:

In [ ]:
tfs = ["CREM", "ZNF263", "FOXA1", "NR3C1"]
for tf in tfs:
    # get motif, analyse
    print(tf)

PS: You can also get all human motifs from the Jaspar database with `human_motifs = jaspar_object.fetch_motifs(collection="CORE", species=["9606"])`, but analysing all of these might take some time.